# Fine-Tuning `Phi-4-mini` mit ElaLoRA

Dieses Notebook zeigt, wie sich das Repository [ElaLoRA](https://github.com/microsoft/ElaLoRA) nutzen lässt, um `microsoft/Phi-4-mini-instruct` mit ElaLoRA zu trainieren. Die Schritte orientieren sich an den Beispielen im Repository.

## Installation
Zunächst werden die benötigten Pakete installiert und das lokale `loralib`-Package eingebunden.

In [ ]:
!pip install transformers datasets accelerate -q
!pip install -e ./loralib


## Modell und Tokenizer laden
Das vortrainierte Modell `microsoft/Phi-4-mini-instruct` wird von Hugging Face geladen.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_id = 'microsoft/Phi-4-mini-instruct'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)


## ElaLoRA in das Modell integrieren
Relevante `nn.Linear`-Schichten werden durch `loralib.SVDLinear` ersetzt und der `RankAllocator` initialisiert.

In [ ]:
import loralib as lora
from loralib.elalora import RankAllocator, SVDLinear

replace_modules = ['q_proj','k_proj','v_proj','o_proj','fc1','fc2']
for name, module in model.named_modules():
    if any(m in name for m in replace_modules) and isinstance(module, torch.nn.Linear):
        parent_name = name.rsplit('.',1)[0]
        parent = model.get_submodule(parent_name)
        new_module = SVDLinear(module.in_features, module.out_features, r=8, lora_alpha=16)
        new_module.weight.data = module.weight.data
        if module.bias is not None:
            new_module.bias.data = module.bias.data
        setattr(parent, name.split('.')[-1], new_module)

lora.mark_only_lora_as_trainable(model)
rankallocator = RankAllocator(
    model,
    lora_r=8,
    target_rank=8,
    init_warmup=300,
    final_warmup=500,
    mask_interval=50,
    beta1=0.85,
    beta2=0.85,
)


## Datensatz vorbereiten
Als Beispiel dient das `xsum`-Datenset für Zusammenfassungen.

In [ ]:
from datasets import load_dataset

dataset = load_dataset('xsum')

max_length = 512

def preprocess_function(examples):
    inputs = examples['document']
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary'], max_length=64, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

processed_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)


## Training konfigurieren und starten
Das Training orientiert sich an `run_summarization_no_trainer.py`.

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir='./phi4-elalora',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    learning_rate=5e-5,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_datasets['train'],
    eval_dataset=processed_datasets['validation'],
    data_collator=data_collator,
)

for step, batch in enumerate(trainer.get_train_dataloader()):
    outputs = trainer.model(**batch)
    loss = outputs.loss
    loss.backward()
    rankallocator.update_and_mask(trainer.model, step)
    trainer.optimizer.step()
    trainer.lr_scheduler.step()
    trainer.optimizer.zero_grad()
    if step > 10:
        break


## Weitere Schritte
Nach dem Training können die LoRA-Gewichte mit `loralib.utils.lora_state_dict` gespeichert werden.